# Biospecies NER LLM pipeline

### This notebook is for using LLMNER package for recognizing and extrating the entities in the biodiversity scientific literatures.

In [1]:
# Import necessary libraries
import os
import sys
#import torch
import csv
import re
import json
import pandas as pd
from datetime import datetime
from llmner import FewShotNer
from llmner.data import AnnotatedDocument, Annotation
from llmner.utils import annotated_document_to_conll


# Set your API keys and environment variables
os.environ["OPENAI_API_BASE"] = "https://api.deepinfra.com/v1/openai"
os.environ["OPENAI_API_KEY"] = "74E9wdOXjnQsR24169Ts8mnSX7D8Ul5T"
#"74E9wdOXjnQsR24169Ts8mnSX7D8Ul5T"

# Define paths to S800 dataset
S800_DIR = "./s800/categorized"
S800_ABSTRACTS_DIR = os.path.join(S800_DIR, "zoo")
S800_ANNOTATIONS_PATH = os.path.join(S800_DIR, "S800-zoo.tsv")
OUTPUT_PATH = "./s800_results/"

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_PATH, exist_ok=True)

In [2]:
# Define few-shot entities
few_shot_entities = {
    "common_name": "The non-scientific name used to identify a species in everyday language. These names often describe physical characteristics, geographic origin, or behavior of the organism.",
    
    "scientific_name": "The formal taxonomic name for a species, including binomial nomenclature, abbreviated forms, genus names alone, species names in taxonomic context, and Latin taxonomic descriptors. Scientific names are the standardized nomenclature used in biology to refer to specific species."
}

few_shot_examples = [
    AnnotatedDocument(
        text="The complete sequence of the 16S rRNA gene of Mycoplasma felis, isolated from cats, was determined.",
        annotations=[
            Annotation(start=53, end=68, label="scientific_name", text="Mycoplasma felis"),
            Annotation(start=80, end=84, label="common_name", text="cats"),
        ],
    ),
    AnnotatedDocument(
        text="Mouse interleukin-2 (IL-2) stimulated the proliferation of mouse and rat cells but human IL-2 stimulated rat cells more effectively than mouse cells.",
        annotations=[
            Annotation(start=0, end=5, label="common_name", text="Mouse"),
            Annotation(start=59, end=64, label="common_name", text="mouse"),
            Annotation(start=69, end=72, label="common_name", text="rat"),
            Annotation(start=99, end=102, label="common_name", text="rat"),
            Annotation(start=129, end=134, label="common_name", text="mouse"),
        ],
    ),
    AnnotatedDocument(
        text="Arabidopsis thaliana and Oryza sativa were compared to study evolutionary differences between dicots and monocots.",
        annotations=[
            Annotation(start=0, end=19, label="scientific_name", text="Arabidopsis thaliana"),
            Annotation(start=24, end=36, label="scientific_name", text="Oryza sativa"),
        ],
    ),
    AnnotatedDocument(
        text="E. coli strains were resistant to ampicillin, while S. aureus showed susceptibility to methicillin.",
        annotations=[
            Annotation(start=0, end=7, label="scientific_name", text="E. coli"),
            Annotation(start=45, end=54, label="scientific_name", text="S. aureus"),
        ],
    ),
    AnnotatedDocument(
    text="The red fox, or Vulpes vulpes, is widespread across the Northern Hemisphere.",
    annotations=[
        Annotation(start=4, end=12, label="common_name", text="red fox"),
        Annotation(start=17, end=31, label="scientific_name", text="Vulpes vulpes"),
        ],
    )
]

In [3]:
import re

def chunk_text(text, max_tokens=512, chunk_overlap=0):
    """
    Langchain-style recursive text splitter without dependencies.
    Tries to split first on \n\n, then \n, then ., then space, then hard cut.
    """
    def split_recursive(text, max_tokens):
        separators = ["\n\n", "\n", r"(?<=[.?!])\s", " "]
        for sep in separators:
            parts = re.split(sep, text) if not sep.startswith("(?") else re.split(sep, text)
            chunks, current = [], []
            total_words = 0

            for part in parts:
                word_count = len(part.split())
                if total_words + word_count <= max_tokens:
                    current.append(part)
                    total_words += word_count
                else:
                    if current:
                        chunks.append(" ".join(current).strip())
                    current = [part]
                    total_words = word_count

            if current:
                chunks.append(" ".join(current).strip())

            # If all chunks are within max_tokens, done
            if all(len(chunk.split()) <= max_tokens for chunk in chunks):
                return chunks

        # fallback: force cut
        words = text.split()
        return [
            " ".join(words[i:i+max_tokens])
            for i in range(0, len(words), max_tokens)
        ]

    base_chunks = split_recursive(text, max_tokens)

    # Apply overlap if needed
    if chunk_overlap > 0:
        overlapped_chunks = []
        for i in range(0, len(base_chunks)):
            current_chunk = base_chunks[i]
            if i > 0:
                previous_chunk = base_chunks[i - 1]
                overlap_words = " ".join(previous_chunk.split()[-chunk_overlap:])
                current_chunk = f"{overlap_words} {current_chunk}"
            overlapped_chunks.append(current_chunk.strip())
        return overlapped_chunks

    return base_chunks


In [22]:
# Initialize the model
model = "meta-llama/Llama-4-Scout-17B-16E-Instruct"
temperature = 0.1
prompting_method = "multi_turn"

# Create the NER model
few_model = FewShotNer(
    model=model,
    temperature=temperature, 
    prompting_method=prompting_method,
    final_message_with_all_entities=True
)

# Contextualize with examples
few_model.contextualize(entities=few_shot_entities, examples=few_shot_examples)

In [23]:
def process_s800_abstracts():
    """Process abstracts from S800 corpus with the given chunk size."""
    results = []
    conll_results = []
    abstract_files = [f for f in os.listdir(S800_ABSTRACTS_DIR) if f.endswith('.txt')]
    print(f"Found {len(abstract_files)} abstract files to process")

    allowed_docs = [
    "species016",
    "species036",
    "species042",
    "species043",
    "species044",
    "species066",
    "species103",
    "species160",
    "species161",
    "species162",
    "species278",
    "species369",
    "species494",
    "species542",
    "species563",
    "species630",
    "species629",
    "species084"
    ]
    file_num = 0
    for filename in abstract_files:
        doc_id = filename[:-4]  
        if doc_id in allowed_docs:
            # Read the abstract content
            with open(os.path.join(S800_ABSTRACTS_DIR, filename), 'r', encoding='utf-8') as f:
                content = f.read()
            print(f"Processing {filename}...")
            file_num += 1
            
            # Chunk the abstract using chunk_text function
            chunks = chunk_text(content, max_tokens=1024, chunk_overlap=0)
            for chunk in enumerate(chunks):
                print('####', chunk)
                
            
            file_results = []
            for i, chunk in enumerate(chunks):
                
                print(f"  Processing chunk {i+1}/{len(chunks)}")
                
                # Get model predictions
                model_output = few_model.predict([chunk])
                annotations_only = model_output[0].annotations
                annotations = model_output[0]
    
                # Convert to CoNLL format using the package function
                conll_format = annotated_document_to_conll(annotations)
                
                # Store results for the whole file
                results.append({
                    'file': filename,
                    'doc_id': doc_id,
                    'content': content,
                    'annotations': annotations.annotations,
                })
                # Store CoNLL format separately
                conll_results.append({
                    'doc_id': doc_id,
                    'conll': conll_format
                })
                
                print(f"Completed file {file_num}/{len(abstract_files)}")
        
    return results, conll_results

In [24]:
results, conll_results = process_s800_abstracts()

Found 100 abstract files to process
Processing species162.txt...
#### (0, 'Constraints on host choice: why do parasitic birds rarely exploit some common potential hosts?  1. Why are some common and apparently suitable resources avoided by potential users? This interesting ecological and evolutionary conundrum is vividly illustrated by obligate brood parasites. Parasitic birds lay their eggs into nests of a wide range of host species, including many rare ones, but do not parasitize some commonly co-occurring potential hosts. 2. Attempts to explain the absence of parasitism in common potential hosts are limited and typically focused on single-factor explanations while ignoring other potential factors. We tested why thrushes Turdus spp. are extremely rarely parasitized by common cuckoos Cuculus canorus despite breeding commonly in sympatry and building the most conspicuous nests among forest-breeding passerines. 3. No single examined factor explained cuckoo avoidance of thrushes. Life-his

100%|███████████████████████████████████████| 1/1 [00:29<00:00, 29.36s/ example]


Completed file 1/100
Processing species084.txt...
#### (0, "Heating up relations between cold fish: competition modifies responses to climate change.  Most predictions about species responses to climate change ignore species interactions. Helland and colleagues (2011) test whether this assumption is valid by evaluating whether ice cover affects competition between brown trout [Salmo trutta (L.)] and Arctic charr [Salvelinus alpines (L.)]. They show that increasing ice cover correlates with lower trout biomass when Arctic charr co-occur, but not in charr's absence. In experiments, charr grew better in the cold, dark environments that typify ice-covered lakes. Decreasing ice cover with warmer winters could mean more trout and fewer charr. More generally, their results provide an excellent example, suggesting that species interactions can strongly modify responses to climate change.")
  Processing chunk 1/1


100%|███████████████████████████████████████| 1/1 [00:18<00:00, 18.75s/ example]


Completed file 2/100
Processing species369.txt...
#### (0, "Predation and patchiness in the tropical litter: do swarm-raiding army ants skim the cream or drain the bottle?  1. Swarm-raiding army ants have long been considered as episodic, catastrophic agents of disturbance in the tropical litter, but few quantitative data exist on their diets, preferences, and, critically, their ability to deplete prey. 2. Here, we provide such data for two common species of swarm raiders broadly sympatric throughout the Neotropics: the iconic Eciton burchellii and the more secretive, less studied Labidus praedator. In Ecuador, Costa Rica, Venezuela and Panama, patches of forest floor were sampled for litter invertebrates immediately before and after army ant raids. These invertebrates have been shown to regulate decomposition and vary 100-fold in local densities across the forest floor. 3. Contrary to Eciton's popular image, only Labidus consistently reduced the biomass of litter invertebrates and onl

...

Completed file 16/100
Processing species103.txt...
#### (0, 'Linking disease and community ecology through behavioural indicators: immunochallenge of white-footed mice and its ecological impacts.  1. Pathogens and immune challenges can induce changes in host phenotype in ways that indirectly impact important community interactions, including those that affect host-pathogen interactions. 2. To explore host behavioural response to immune challenge, we exposed wild white-footed mice (Peromyscus leucopus) to an immunogen from an endemic, zoonotic pathogen, the spirochete Borrelia burgdorferi. White-footed mice are a major reservoir host of Lyme disease (LD) spirochetes in northeastern USA and an abundant member of forest communities. The activity patterns, foraging behaviour, and space use of white-footed mice have implications for population growth rates of community members upon which mice incidentally prey (i.e. gypsy moths and native thrushes), as well as potentially determining host-v

100%|███████████████████████████████████████| 1/1 [00:30<00:00, 30.32s/ example]


Completed file 17/100
Processing species160.txt...
#### (0, "Predicting the optimal prey group size from predator hunting behaviour. 1. How group size affects predator attack and success rate, and so prey vulnerability, is important in determining the nonlethal consequences of predation risk on animal populations and communities. Theory predicts that both predator attack success rate and the dilution effect decline exponentially with group size and that selection generates optimal group sizes at a 'risk threshold' above which antipredation benefits are outweighed by costs, such as those owing to higher attack rates. 2. We examined whether flock size risk thresholds for attack rate, success rate or dilution differed, and therefore whether the strength of selection for group size differed for these three factors, using a system of redshank Tringa totanus flocks being hunted by Eurasian sparrowhawks Accipiter nisus. We also asked which of the three thresholds, on their own or in combinati

100%|███████████████████████████████████████| 1/1 [00:22<00:00, 22.31s/ example]

Completed file 18/100


In [26]:
# Save to CSV with the specific format requested
with open('chunk_1024_llama4_allowed_docs_temp1.csv', 'w', newline='', encoding='utf-8') as f:
    # Define the fields as requested
    fieldnames = ['file', 'doc_id', 'content', 'Annotation Text', 'start', 'end', 'label']
    
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    
    for result in results:
        # Get the base information
        base_info = {
            'file': result['file'],
            'doc_id': result['doc_id'],
            'content': result['content'],
        }
        
        # Handle annotations - assuming annotations is a list of objects with text, start, end, and label properties
        if 'annotations' in result and result['annotations']:
            for annotation in result['annotations']:
                row = base_info.copy()  # Start with the base information
                
                # Add annotation details - handle different annotation formats
                # Try direct dictionary access first
                try:
                    row['Annotation Text'] = annotation['text'] if 'text' in annotation else ''
                    row['start'] = annotation['start'] if 'start' in annotation else ''
                    row['end'] = annotation['end'] if 'end' in annotation else ''
                    row['label'] = annotation['label'] if 'label' in annotation else ''
                except (TypeError, KeyError):
                    # If that fails, try attribute access
                    try:
                        row['Annotation Text'] = getattr(annotation, 'text', '')
                        row['start'] = getattr(annotation, 'start', '')
                        row['end'] = getattr(annotation, 'end', '')
                        row['label'] = getattr(annotation, 'label', '')
                    except (AttributeError, TypeError):
                        # Last resort: convert to string
                        row['Annotation Text'] = str(annotation)
                        row['start'] = ''
                        row['end'] = ''
                        row['label'] = ''
                
                writer.writerow(row)
        else:
            # If no annotations, write just the base info
            row = base_info.copy()
            row['Annotation Text'] = ''
            row['start'] = ''
            row['end'] = ''
            row['label'] = ''
            writer.writerow(row)

print("Results saved to all_results.csv")

print('hello')

Results saved to all_results.csv
hello
